In [1]:
from automol.datasets import *
config_yaml = 'qm9_dataset_test_config.yaml'

In [4]:
d = Dataset.from_input(config_yaml)

In [15]:
d.data['smiles'][0:10]

['O=C([O-])C12C[NH2+]C1C2',
 'CN1CC1C#CCO',
 'OCC1COCOC=N1',
 'COCCC(C#N)C=O',
 'C1CC2OC2C2CN2C1',
 'CC12C3OC1C2NC3=O',
 'C#CCN(C)C=O',
 'CC1OC2CC1C2(C)C',
 'O=C1NC2CCC2O1',
 'CC1(C(=O)C=O)CCC1']

In [16]:
d.data['homo'][0:10]

[-0.1795,
 -0.2284,
 -0.2521,
 -0.2701,
 -0.2201,
 -0.227,
 -0.2516,
 -0.231,
 -0.2575,
 -0.2452]

In [17]:
'smiles' in d.data

True